#Pythonで学ぶ画像認識　付録A PyTorchの基礎

本付録では本書で使う深層学習フレームワークのPyTorchについて、本書を読み進める上で必要となる最低限の知識を解説します。PyTorchではNumPyと同じように、複数の数値を1つにまとめた多次元配列を使って処理を進めます。多次元配列の使い方や多次元配列に適用する関数はNumPyと類似する部分が多くあります。NumPyの使い方をご存じの方はそれらの処理を思い浮かべながら共通する部分を確認していただければと思います。

##モジュールのインポート

In [1]:
import torch
import torch.nn.functional as F
from torch import nn

##テンソルとテンソルの操作

###テンソルとは

####テンソルの生成

まずはPyTorchの多次元配列の基礎と多次元配列を使った簡単な処理を解説します。PyTorchでは多次元配列のことを**テンソル（Tensor）**と呼び、`Tensor`クラスで定義しています。以下にテンソルの生成例を示します。

In [2]:
t1 = torch.tensor([1, 2, 3, 4])
t2 = torch.zeros((32, 3, 128, 128))

print(f't1 = {t1}, t1.shape = {t1.shape}')
print(f't2.shape = {t2.shape}')

t1 = tensor([1, 2, 3, 4]), t1.shape = torch.Size([4])
t2.shape = torch.Size([32, 3, 128, 128])


`t1`は`tensor`関数にリストを渡すことで生成されています。この生成結果は第0軸が4次元で1から4の値を持つテンソルです。`t2`は`zeros`関数により生成されています。`zeros`関数は各軸の次元を指定することで、その形を持つ全ての値が0のテンソルを生成します。このようにテンソルは任意の軸数および次元を持つことができます。テンソルの形は`Tensor`クラスのインスタンスが持つ`shape`変数で確認できます。

####テンソルのGPUへの転送

テンソルは通常だとCPUで処理するためにメインメモリ上に生成されますが、GPUメモリ上に生成したり、後からGPUメモリに転送したりできます。以下にGPUメモリにテンソルを配置するための例を示します。

In [3]:
t1 = torch.tensor([1, 2, 3, 4], device='cuda')

t2 = torch.tensor([1, 2, 3, 4])
t2 = t2.to('cuda')

`t1`は生成時に`device`引数に`'cuda'`を指定しているので最初からGPUメモリ上に生成されます。一方で`t2`はまずメインメモリ上に生成され、`to`関数によりGPUメモリに転送されます。このようにテンソルをGPUメモリに配置してGPUで処理することにより高速に処理できるようになります。ただし、メインメモリと比較してGPUメモリは容量が限られていることが多いので、GPUメモリに配置するテンソルの数と大きさに気をつける必要があります。

####Python演算子を使ったテンソルの演算

Pythonの演算子を使ってテンソルの演算をした場合、演算は要素ごとに行われます。以下にPython演算子によるテンソルの演算とその結果を示します。

In [4]:
t1 = torch.tensor([1, 2, 3, 4])
t2 = torch.tensor([2, 4, 6, 8])

t3 = t1 + t2
t4 = t1 ** 2

print(f't3 = {t3}')
print(f't4 = {t4}')

t3 = tensor([ 3,  6,  9, 12])
t4 = tensor([ 1,  4,  9, 16])


`t3`は`t1`と`t2`の要素毎の加算により得られ、`t4`は`t1`の要素毎の2乗により得られていることがわかります。このような要素毎の演算をするため、Python演算子を使った2つのテンソルの演算では2つのテンソルは基本的に同じ形である必要があります。ただし、後ほど解説するブロードキャストが可能な条件に当てはまれば、異なる形のテンソル同士で演算可能です。

以上がテンソルの解説になります。`Tensor`クラスにはテンソルを処理するための様々な関数が実装されています。次はそれらの中から頻繁に使うものをいくつか解説します。

###テンソルを処理する関数

####view関数によるテンソルの形の変更

PyTorchでデータを処理する際にテンソルの形を変更することが多くあります。そのようなときは`view`関数を使用します。以下に`view`関数の使用例と形を変更した結果を示します。

In [5]:
t1 = torch.tensor([1, 2, 3, 4])
t1 = t1.view(2, 2)

t2 = torch.tensor([1, 2, 3, 4, 5, 6])
t2 = t2.view(2, -1)

print(f't1.shape = {t1.shape}')
print(f't2.shape = {t2.shape}')

t1.shape = torch.Size([2, 2])
t2.shape = torch.Size([2, 3])


`t1`は元々`[4]`の形のテンソルでしたが、`view`関数により`[2, 2]`の形に変更されています。`t2`の形は`view`関数により`[6]`から`[2, 3]`に変更されています。`view`関数には1つだけ`-1`を渡すことができ、`-1`を指定された軸の次元は、元のテンソルの大きさと`view`関数に渡されたその他の軸の次元から自動的に計算されます。

`view`関数は元のテンソルのデータをメモリ上で複製せずに、見かけ上の形を変更します。このような理由からメモリ上のデータ配置と見かけ上の形で整合性が取れないときは`view`関数を使えません。このようなときは`reshape`関数を使います。`reshape`関数は前述のような整合性がとれないときは、データを複製して形を変更します。多くの場合で`view`関数を使うことができるので、`view`関数でエラーが出るときのみ`reshape`関数を使えば問題ありません。

####transpose関数とpermute関数による軸の並び替え

`view`関数と同様にテンソルの処理で頻繁に登場するのが軸の順番を並び替える`transpose`関数や`permute`関数です。軸の並び替えというと想像しにくいですが、行列の行と列を入れ替える操作をより多くの軸で行うイメージです。`transpose`関数は任意の2軸を入れ替えます。一方で`permute`関数は`transpose`関数を拡張した関数で、全ての軸を一度に並べ替えることができます。以下に`transpose`関数と`permute`関数の使用例と軸を並び替えた結果を示します。

In [6]:
t1 = torch.zeros((32, 3, 128, 128))
t1 = t1.transpose(0, 2)

t2 = torch.zeros((32, 3, 128, 128))
t2 = t2.permute(2, 0, 3, 1)

print(f't1.shape = {t1.shape}')
print(f't2.shape = {t2.shape}')

t1.shape = torch.Size([128, 3, 32, 128])
t2.shape = torch.Size([128, 32, 128, 3])


`transpose`関数で第0軸と第2軸を入れ替えた結果、`[32, 3, 128, 128]`の形であったテンソル`t1`が`[128, 3, 32, 128]`の形になります。`permute`関数は軸の順番が第2軸、第0軸、第3軸、第1軸となるよう並べ替えており、軸を並べ替えられた`t2`は`[128, 32, 128, 3]`の形になります。

####cat関数とstack関数による複数テンソルの連結

`view`関数や`transpose`関数は1つのテンソルに対する操作でしたが、複数のテンソルを組み合わせる操作が必要になることも多くあります。そのようなときに使用するのが`cat`関数と`stack`関数です。`cat`関数はテンソルが持つ既存の軸の1つで複数のテンソルを連結します。一方で`stack`関数は新しく軸を追加して、その軸で複数のテンソルを連結します。以下に`cat`関数と`stack`関数の使用例と複数のテンソルを連結した結果を示します。

In [7]:
t1 = torch.tensor([1, 2, 3, 4, 5, 6]).view(2, 3)
t2 = torch.tensor([7, 8, 9]).view(1, 3)
t3 = torch.cat((t1, t2))

t4 = torch.tensor([1, 2, 3])
t5 = torch.tensor([4, 5, 6])
t6 = torch.stack((t4, t5), dim=1)

print(f't3 = {t3}')
print(f't6 = {t6}')

t3 = tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])
t6 = tensor([[1, 4],
        [2, 5],
        [3, 6]])


`cat`関数は`[2, 3]`の形の`t1`と`[1, 3]`の形の`t2`を第0軸で連結して、`[3, 3]`の形の`t3`を生成しています。`cat`関数はデフォルトで第0軸で連結しますが、連結する軸を`dim`引数で指定することもできます。`cat`関数を使うときは、連結されるテンソルは`dim`引数で指定する軸以外は同じ次元である必要があります。`stack`関数は`[3]`の形の`t4`と`[3]`の形の`t5`に第1軸を追加してその軸で連結し、`[3, 2]`の形の`t6`を生成しています。`stack`関数を使うときは、連結されるテンソルは全て同じ形である必要があります。

###インデクシングによるテンソルの要素の抽出

データを処理していると、テンソルから一部の要素を抽出する必要があるときがあります。そのようなときは、Pythonのリストと同じようにインデクシングにより要素を抽出できます。ただし、`Tensor`クラスにはより高度なインデクシングの方法が実装されています。以下にインデクシングの例とその結果を示します。

In [8]:
t1 = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9]).view(3, 3)
t2 = t1[[0, 1]]
t3 = t1[:, [0, 2]]
t4 = t1[[0, 2, 1], [1, 2, 1]]
t5 = t1[[True, False, False]]
t6 = t1[t1 % 2 == 0]

print(f't2 = {t2}')
print(f't3 = {t3}')
print(f't4 = {t4}')
print(f't5 = {t5}')
print(f't6 = {t6}')

t2 = tensor([[1, 2, 3],
        [4, 5, 6]])
t3 = tensor([[1, 3],
        [4, 6],
        [7, 9]])
t4 = tensor([2, 9, 5])
t5 = tensor([[1, 2, 3]])
t6 = tensor([2, 4, 6, 8])


インデクシングの1つの方法は抽出したい次元を指定する方法です。`t2`や`t3`はそれぞれ`t1`の第0軸と第1軸の次元を複数のインデックスを使って指定し、その次元の値を抽出することで得られるテンソルになります。このように1つの軸に対して複数のインデックスを指定することで、複数の次元を抽出した新しいテンソルを生成できます。一方で`t4`は`t1`の2つの軸に同時にインデックスを指定して値を抽出したテンソルになります。このように複数の軸に同時にインデックスを指定した場合、インデックスを座標のように使った値の抽出ができます。`t4`の例では2軸を持つテンソルである`t1`から0行1列目、2行2列目および1行1列目の3つの値を抽出したテンソルになります。

インデクシングのもう1つの方法は真偽値を使う方法です。`t5`の例では`t1`の第0軸に対して抽出対象の次元に`True`を、そうでない次元に`False`を設定して値を抽出しています。このように1つの軸の各次元に真偽値を設定することによって、その軸の必要な次元を抽出することができます。また、`t6`の例のように`t1`の全ての要素に対して真偽値を設定し、必要な要素を取り出すこともできます。`t6`は`t1`の要素が偶数かどうかを表す真偽値をインデックスとして使い、抽出されたテンソルです。このようなインデクシングにより、`t1`から偶数のみを抽出したテンソルを得られています。

###ブロードキャストを使った演算

Python演算子を使ったテンソルの計算のところで解説したように、要素毎の演算において異なる形を持つテンソル同士がある一定の条件を満たすとき、それらを使って演算を行うことができます。これは一方のテンソルの形が他方に合わせて自動的に拡張して解釈されるためで、**ブロードキャスト（broadcast）**と呼ばれます。ブロードキャストが起きる条件は以下になります。

* 2つのテンソルの軸数が1以上である
* 2つのテンソルを最終軸から比較した場合、各軸の次元が同じであるか、どちらかが1であるか、どちらかの軸が存在しない

以下にブロードキャストが起きる例とその演算結果を示します。

In [9]:
t1 = torch.tensor([1, 2]).view(2, 1)
t2 = torch.tensor([3, 4, 5])
# t1 = [[1],         ->      [[1, 1, 1],
#       [2]]     broadcast    [2, 2, 2]]
# t2 =  [3, 4, 5]    ->      [[3, 4, 5],
#                             [3, 4, 5]]
t3 = t1 + t2

t4 = torch.tensor([1, 2, 3, 4, 5, 6]).view(3, 2)
t5 = torch.tensor([3, 4])
# t4 = [[1, 2],
#       [3, 4],
#       [5, 6]]  broadcast    
# t5 =  [3, 4]       ->      [[3, 4],
#                             [3, 4],
#                             [3, 4]]
t6 = t4 + t5

print(f't3 = {t3}')
print(f't6 = {t6}')

t3 = tensor([[4, 5, 6],
        [5, 6, 7]])
t6 = tensor([[ 4,  6],
        [ 6,  8],
        [ 8, 10]])


`t1`と`t2`の加算では`t1`の形`[2, 1]`と`t2`の形`[3]`が最終軸から比較されます。`t1`の最終軸の次元は1、`t2`の最終軸の次元は3であるので、`t1`の軸が拡張して解釈され、最終軸の次元が3であるかのように扱われます。`t1`の1つ前の軸の次元は2、`t2`は1つ前の軸がないので、`t2`の軸が拡張して解釈され、第0軸が追加されてその次元が2のように扱われます。その結果、`[2, 3]`の形の`t3`が得られます。`t4`と`t5`の加算では`t4`の形`[3, 2]`と`t5`の形`[2]`が最終軸から比較されます。`t4`と`t5`の最終軸の次元はともに2であるので、なにもせずに1つ前の軸が比較されます。`t4`の1つ前の軸の次元は3、`t5`は1つ前の軸がないので、`t5`の軸が拡張して解釈され、第0軸が追加されてその次元が3のように扱われます。その結果、`[3, 2]`の形の`t6`が得られます。

以上がテンソルに関する解説になります。次はPyTorchを使ったモデルの実装について解説します。

##モジュール

PyTorchではパラメータを持つモデルを実装する際に決められたやり方に従って実装する必要があります。そのやり方に従うことによって、モデルが持つパラメータの抽出やパラメータのGPUへの転送、モデルを学習に使うのか評価に使うのかの切り替えなどの処理を簡単に実行することができます。以下ではその実装方法について解説します。

###モジュールとは

####多クラスロジスティック回帰のPyTorchを使った実装

PyTorchでは1つ1つのモデルをクラスで定義し、PyTorchの`Module`クラスを継承して実装します。Pythonで`import`して使用するモジュールとPyTorchのモジュールは名称は同じですが異なる概念なので注意してください。以下に多クラスロジスティック回帰モデルを`Module`クラスを継承して実装した例を示します。

In [10]:
class MultiClassLogisticRegression(nn.Module):
    '''
    多クラスロジスティック回帰
    dim_input  : 入力次元
    num_classes: 分類対象の物体クラス数
    '''
    def __init__(self, dim_input: int, num_classes: int):
        super().__init__()
        
        self.linear = nn.Linear(dim_input, num_classes)

    '''
    順伝播関数
    x: 入力データ, [バッチサイズ, 入力次元]
    '''
    def forward(self, x: torch.Tensor):
        l = self.linear(x)
        y = l.softmax(dim=1)

        return y

`Module`クラスを継承したモデルクラスのコンストラクタでは、まず親クラスのコンストラクタを呼ぶ必要があります。そのあとにモデルに必要なものを用意します。

PyTorchには`Tensor`クラスを拡張した`Parameter`クラスがあり、そのクラスがパラメータの実装になります。しかし、自分で実装するモデルクラスのコンストラクタの中で`Parameter`クラスのインスタンスを生成することはあまりありません。なぜならPyTorchには深層学習で必要となる多くの処理が既にクラスとして実装されており、それらの中でそれぞれに必要なパラメータが生成されるからです。例えば線形関数は`Linear`クラスとして実装されており、上記の`MultiClassLogisticRegression`クラスではそれを使用しています。`Linear`クラスもまた`Module`クラスを継承したクラスとなっており、このクラスのコンストラクタの中で重みやバイアスのパラメータが用意されています。

モデルの実装でコンストラクタの他にもう1つ必要になるのが順伝播を行う`forward`関数です。`Module`クラスには`__call__`関数が定義されており、`Module`クラスのインスタンスを関数と同じように使えます。`__call__`関数を呼び出すと`forward`関数が呼ばれ、順伝播が行われます。上記の`forward`関数では線形関数の順伝播によりロジット`l`を得た後、`Tensor`クラスが持つ`softmax`関数を第1軸に適用することで予測確率を得ています。

####多クラスロジスティック回帰モデルの使用例

以下に実装した多クラスロジスティック回帰モデルの使用例を示します。


In [11]:
model = MultiClassLogisticRegression(32 * 32 * 3, 10)

# 学習モードに設定
model.train()

# 評価(推論)モードに設定
model.eval()

x = torch.normal(0, 1, size=(1, 32 * 32 * 3))
y = model(x)

for name, parameter in model.named_parameters():
    print(f'{name}: shape = {parameter.shape}')

linear.weight: shape = torch.Size([10, 3072])
linear.bias: shape = torch.Size([10])


`Module`クラスには`train`関数と`eval`関数が実装されており、モデルの学習モードと評価モードを切り替えられるようになっています。今回の多クラスロジスティック回帰モデルでは学習モードと評価モードに違いはありませんが、モデルの中で学習と評価で異なる処理が必要となることもあるため、このような関数を用意して学習モードと評価モードを簡単に切り替えられるようになっています。

`Module`クラスに実装されている`named_parameters`関数を使うことで、モデルが持つパラメータをその名前とともに抽出することができます。上記コードの実行結果には`MultiClassLogisticRegression`クラスのパラメータの名前と形が示されています。`MultiClassLogisticRegression`クラスで`Linear`クラスのインスタンスは`linear`変数に登録されており、`Linear`クラスで重みは`weight`変数に、バイアスは`bias`変数に登録されているため、パラメータの名前は上記結果のようになります。

以上がモジュールの解説になります。次は複数のモジュールを1つにまとめるために用意されているクラスを解説します。

###`Sequential`クラスと`ModuleList`クラス

PyTorchには複数の`Module`クラスのインスタンスを1つにまとめるための`Sequential`クラスおよび`ModuleList`クラスが用意されています。`Sequential`クラスは複数の処理を直列にまとめて適用するためのクラスです。一方で`ModuleList`クラスはリストのように複数の処理をリストにまとめて保持しておくためのクラスです。ここでは`Sequential`クラスと`ModuleList`クラスを使いながら2つのクラスがどのように異なるかを確認します。

####Sequentialクラスを使ったモデルの実装

以下に`Sequential`クラスを使った順伝播型ニューラルネットワークの実装例を示します。

In [12]:
class FNNSequential(nn.Module):
    '''
    順伝播型ニューラルネットワーク
    dim_input  : 入力次元
    num_classes: 分類対象の物体クラス数
    '''
    def __init__(self, dim_input: int, num_classes: int):
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(dim_input, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, num_classes)
        )

    '''
    順伝播関数
    x: 入力データ, [バッチサイズ, 入力次元]
    '''
    def forward(self, x):
        l = self.layers(x)
        y = l.softmax(dim=1)

        return y


`FNNSequential`クラスのコンストラクタの中で`Sequential`クラスを使用しています。`Sequential`クラスのコンストラクタには適用したい処理のクラスインスタンスを渡します。ここでは線形関数である`Linear`クラスとReLU関数である`ReLU`クラスのインスタンスを渡しています。これらの処理は引数で渡した順番で直列に適用されます。`forward`関数の実装に示すように、順伝播を行うには`Sequential`クラスのインスタンスに入力を渡します。得られる結果は`Sequential`クラスのコンストラクタに渡した全ての処理を適用したものになります。

####ModuleListクラスを使ったモデルの実装

以下に`ModuleList`クラスを使った順伝播型ニューラルネットワークの実装例を示します。

In [13]:
class FNNModuleList(nn.Module):
    '''
    順伝播型ニューラルネットワーク
    dim_input  : 入力次元
    num_classes: 分類対象の物体クラス数
    '''
    def __init__(self, dim_input: int, num_classes: int):
        super().__init__()
        
        layers = [nn.Linear(dim_input, 256)]
        layers += [nn.Linear(256, 256) for _ in range(2)]
        layers.append(nn.Linear(256, num_classes))
        self.layers = nn.ModuleList(layers)

    '''
    順伝播関数
    x: 入力データ, [バッチサイズ, 入力次元]
    '''
    def forward(self, x):
        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
        l = self.layers[-1](x)
        y = l.softmax(dim=1)
        
        return y

`ModuleList`クラスのコンストラクタには必要な処理のクラスを集めたリストを渡します。このように複数の処理をまとめたリストを使う場合にはPythonクラスのリストではなく`ModuleList`クラスを使う必要があります。`ModueList`クラスを使わずにPythonのリストをそのままクラス変数に登録してしまった場合、モデルが持つパラメータの管理がうまくできなくなります。`ModueList`クラスの使い方はPythonリストと同じで、順伝播時には`for`ループなどを使って要素を抽出して使います。

####`Sequential`クラスと`ModuleList`クラスを使ったFNNモデルの使用例

In [14]:
model_sequential = FNNSequential(32 * 32 * 3, 10)
model_modulelist = FNNModuleList(32 * 32 * 3, 10)

model_sequential.eval()
model_modulelist.eval()

x = torch.normal(0, 1, size=(1, 32 * 32 * 3))
y_sequential = model_sequential(x)
y_modulelist = model_modulelist(x)

print(f'y_sequential = {y_sequential}')
print(f'y_modulelist = {y_modulelist}')

y_sequential = tensor([[0.1015, 0.0933, 0.0947, 0.0955, 0.1072, 0.0904, 0.1054, 0.1059, 0.0952,
         0.1109]], grad_fn=<SoftmaxBackward0>)
y_modulelist = tensor([[0.0947, 0.0904, 0.1014, 0.0958, 0.0994, 0.1121, 0.1010, 0.0981, 0.0946,
         0.1125]], grad_fn=<SoftmaxBackward0>)


上記に示すように、`Sequential`クラスを使った場合と`ModuleList`クラスを使った場合で、モデルの外から見た違いはありません。では2つのクラスをモデルの中でどのように使い分けるかというと、`Sequential`クラスはまとまった複数の処理を1つの処理として適用するときによく使われます。例えば線形関数とReLU関数は1セットの処理と考えられるので、`Sequential`クラスで`Linear`クラスと`ReLU`クラスをまとめます。一方で`ModuleList`クラスは同じ処理を複数回適用するようなときに使われます。例えば`Sequential`クラスでまとめた線形関数とReLU関数を3回適用したい場合などは、`Sequential`クラスのインスタンスを3つ`ModuleList`クラスでまとめます。

以上がPyTorchのモジュールの解説になります。最後にPyTorchの誤差逆伝播のために実装された自動微分について解説します。

##自動微分

PyTorchでは誤差逆伝播を簡単に行うために自動微分という仕組みが実装されています。第3.1節で実装した多クラスロジスティック回帰モデルでは勾配の計算を実装してパラメータの更新を行いましたが、自動微分があるPyTorchを使えば勾配の計算を実装する必要はありません。

第3.2節で解説したように、DNNはグラフ構造として捉えられます。順伝播ではグラフを順方向に辿って出力を計算し、誤差逆伝播では順伝播とは逆方向に辿って勾配を計算します。また、誤差逆伝播では連鎖率により現在の処理の勾配と逆伝播されてきた勾配の積を計算すれば、目的関数のパラメータ方向の勾配を計算できました。これらを考慮すると、順伝播でグラフ構造を把握し、かつ個々の処理で勾配の計算が実装されていれば誤差逆伝播を行えることになります。

PyTorchではこのような順伝播時のグラフ構造の記録と個々の処理の勾配計算の実装により自動微分を実現しています。PyTorchでは入力から出力までで、適用された関数や計算の途中結果がグラフを構築するように記録され、誤差逆伝播時にはそれを逆にたどることで全てのパラメータの勾配が計算されます。グラフの記録を漏れなく行うためには、テンソルに対する処理を全てPyTorchの関数で行う必要があります。PyTorchの関数にない独自の処理を行いたい場合には、PyTorchの関数の設計方針に従って関数を実装することも可能です。しかし、多くの場合はPyTorchの関数を組み合わせれば目的の処理を実現できます。

以下に自動微分を使った誤差逆伝播の例を示します。


In [15]:
linear = nn.Linear(32 * 32 * 3, 10)

# 入力とラベルの用意
x = torch.normal(0, 1, size=(1, 32 * 32 * 3))
y = torch.tensor([0])

# 目的関数(交差エントロピー誤差)の計算
y_pred = linear(x)
loss = F.cross_entropy(y_pred, y)

# 誤差逆伝播
loss.backward()

print(f'linear.weight.grad = {linear.weight.grad}')
print(f'linear.bias.grad = {linear.bias.grad}')

linear.weight.grad = tensor([[ 0.0814,  0.1924, -1.3381,  ..., -0.5630, -0.7319,  0.1235],
        [-0.0034, -0.0081,  0.0564,  ...,  0.0237,  0.0308, -0.0052],
        [-0.0124, -0.0294,  0.2045,  ...,  0.0860,  0.1119, -0.0189],
        ...,
        [-0.0074, -0.0174,  0.1209,  ...,  0.0509,  0.0661, -0.0112],
        [-0.0208, -0.0491,  0.3417,  ...,  0.1438,  0.1869, -0.0315],
        [-0.0080, -0.0189,  0.1316,  ...,  0.0554,  0.0720, -0.0121]])
linear.bias.grad = tensor([-0.8701,  0.0367,  0.1330,  0.1623,  0.0346,  0.0345,  0.0828,  0.0786,
         0.2222,  0.0856])


誤差逆伝播を行う際は、まずは入力に対して順伝播を行い、目的関数を計算します。あとは`Tensor`クラスに実装された`backward`関数を呼ぶだけで誤差逆伝播が完了します。

勾配は上記に示すように`Parameter`クラスのインスタンスの`grad`変数に格納されています。パラメータの更新はこの勾配を使って行われます。

以上がPyTorchの基礎の解説になります。